# Joining Data in SQL

Now that you've learned the basics of SQL, it's time to supercharge your queries using joins and relational set theory. In this notebook, you'll learn all about the power of joining tables while exploring interesting features of countries and their cities throughout the world. You will master inner and outer joins, as well as self joins, semi joins, anti joins and cross joins—fundamental tools in any PostgreSQL wizard's toolbox. Never fear set theory again after learning all about unions, intersections, and except clauses through easy-to-understand diagrams and examples. Lastly, you'll be introduced to the challenging topic of subqueries. You will be able to visually grasp these ideas by using Venn diagrams and other linking illustrations.

In [1]:
! postgres --version

postgres (PostgreSQL) 11.3


In [2]:
import pandas as pd
import psycopg2 as pg

In [ ]:
# To setup the database:
# In pgAdmin (http://127.0.0.1:50822/browser/): 
# - create a user the same as for macos user
# - create films database with the user as the owner
# In terminal in the countries2 folder:
# > psql -U ksatola countries < 'countries.sql''

In [3]:
conn = pg.connect(database="countries",user="ksatola", password="ksroot")

In [4]:
sql ='''
SELECT * 
FROM countries 
LIMIT 5
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,Afganistan/Afqanestan,Islamic Emirate,Kabul,69.17610,34.5228
1,NLD,Netherlands,Europe,Western Europe,41526.0,1581.0,Nederland,Constitutional Monarchy,Amsterdam,4.89095,52.3738
2,ALB,Albania,Europe,Southern Europe,28748.0,1912.0,Shqiperia,Republic,Tirane,19.81720,41.3317
3,DZA,Algeria,Africa,Northern Africa,2381740.0,1962.0,Al-Jazair/Algerie,Republic,Algiers,3.05097,36.7397
4,ASM,American Samoa,Oceania,Polynesia,199.0,NaN,Amerika Samoa,US Territory,Pago Pago,-170.69100,-14.2846


## Inner join

<img src="images/sql-join-inner.png" alt="" style="width: 250px;"/>

You'll be working with the `countries` database containing information about the most populous world cities as well as country-level economic data, population data, and geographic data. This countries database also contains information on languages spoken in each country.

Here is the basic syntax for an INNER JOIN, here including all columns in both tables:

In [5]:
sql ='''
SELECT * 
FROM cities
  -- 1. Inner join to countries
  INNER JOIN countries
    -- 2. Match on the country codes
    ON cities.country_code = countries.code;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,country_code,city_proper_pop,metroarea_pop,urbanarea_pop,code,name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
0,Abidjan,CIV,4765000.0,NaN,4765000.0,CIV,Cote d'Ivoire,Africa,Western Africa,322463.0,1960,Cote dIvoire,Republic,Yamoussoukro,-4.03050,5.33200
1,Abu Dhabi,ARE,1145000.0,NaN,1145000.0,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971,Al-Imarat al-´Arabiya al-Muttahida,Emirate Federation,Abu Dhabi,54.37050,24.47640
2,Abuja,NGA,1235880.0,6000000.0,1235880.0,NGA,Nigeria,Africa,Western Africa,923768.0,1960,Nigeria,Federal Republic,Abuja,7.48906,9.05804
3,Accra,GHA,2070460.0,4010050.0,2070460.0,GHA,Ghana,Africa,Western Africa,238533.0,1957,Ghana,Republic,Accra,-0.20795,5.57045
4,Addis Ababa,ETH,3103670.0,4567860.0,3103670.0,ETH,Ethiopia,Africa,Eastern Africa,1104300.0,-1000,YeItyop´iya,Republic,Addis Ababa,38.74680,9.02274


In [8]:
sql ='''
-- 1. Select name fields (with alias) and region 
SELECT cities.name AS city,
countries.name AS country,
countries.region
FROM cities
INNER JOIN countries
ON cities.country_code = countries.code
'''
df = pd.read_sql(sql, conn)
df.head()

,city,country,region
0,Abidjan,Cote d'Ivoire,Western Africa
1,Abu Dhabi,United Arab Emirates,Middle East
2,Abuja,Nigeria,Western Africa
3,Accra,Ghana,Western Africa
4,Addis Ababa,Ethiopia,Eastern Africa


## Inner join (2)

Instead of writing the full table name, you can use table aliasing as a shortcut. For tables you also use `AS` to add the alias immediately after the table name with a space. Check out the aliasing of cities and countries below.

Notice that to select a field in your query that appears in multiple tables, you'll need to identify which table/table alias you're referring to by using a `.` in your `SELECT` statement.

You'll now explore a way to get data from both the countries and economies tables to examine the inflation rate for both 2010 and 2015.

Sometimes it's easier to write SQL code out of order: you write the `SELECT` statement after you've done the `JOIN`.

- Join the tables `countries` (left) and `economies` (right) aliasing countries AS `c` and economies AS `e`.
- Specify the field to match the tables ON.
- From this join, SELECT:
    - c.code, aliased as country_code.
    - name, year, and inflation_rate, not aliased.

In [9]:
sql ='''
-- 3. Select fields with aliases
SELECT c.code AS country_code, c.name, e.year, inflation_rate
FROM countries AS c
  -- 1. Join to economies (alias e)
  INNER JOIN economies AS e
    -- 2. Match on code
    ON c.code = e.code;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,name,year,inflation_rate
0,AFG,Afghanistan,2010,2.179
1,AFG,Afghanistan,2015,-1.549
2,AGO,Angola,2010,14.480
3,AGO,Angola,2015,10.287
4,ALB,Albania,2010,3.605


## Inner join (3)

The ability to combine multiple joins in a single query is a powerful feature of SQL, e.g:

As you can see here it becomes tedious to continually write long table names in joins. This is when it becomes useful to alias each table using the first letter of its name (e.g. countries AS c)! It is standard practice to alias in this way and, if you choose to alias tables or are asked to specifically for an exercise in this course, you should follow this protocol.

Now, for each country, you want to get the country name, its region, and the fertility rate and unemployment rate for both 2010 and 2015.

Note that results should work throughout this course with or without table aliasing unless specified differently.

- Inner join countries (left) and populations (right) on the code and country_code fields respectively.
- Alias countries AS c and populations AS p.
- Select code, name, and region from countries and also select year and fertility_rate from populations (5 fields in total).

In [10]:
sql ='''
-- 4. Select fields
SELECT c.code, c.name, c.region, p.year, p.fertility_rate
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join with populations (as p)
  INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,region,year,fertility_rate
0,ABW,Aruba,Caribbean,2010,1.704
1,ABW,Aruba,Caribbean,2015,1.647
2,AFG,Afghanistan,Southern and Central Asia,2010,5.746
3,AFG,Afghanistan,Southern and Central Asia,2015,4.653
4,AGO,Angola,Central Africa,2010,6.416


- Add an additional inner join with economies to your previous query by joining on code.
- Include the unemployment_rate column that became available through joining with economies.
- Note that year appears in both populations and economies, so you have to explicitly use e.year instead of year as you did before.

In [11]:
sql ='''
-- 6. Select fields
SELECT c.code, name, region, e.year, fertility_rate, e.unemployment_rate
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join to populations (as p)
  INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
  -- 4. Join to economies (as e)
  INNER JOIN economies AS e
    -- 5. Match on country code
    ON c.code = e.code;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,region,year,fertility_rate,unemployment_rate
0,AFG,Afghanistan,Southern and Central Asia,2015,5.746,NaN
1,AFG,Afghanistan,Southern and Central Asia,2010,5.746,NaN
2,AFG,Afghanistan,Southern and Central Asia,2015,4.653,NaN
3,AFG,Afghanistan,Southern and Central Asia,2010,4.653,NaN
4,AGO,Angola,Central Africa,2015,6.416,NaN


- The trouble with doing your last join on c.code = e.code and not also including year is that e.g. the 2010 value for fertility_rate is also paired with the 2015 value for unemployment_rate.
- Fix your previous query: in your last ON clause, use AND to add an additional joining condition. In addition to joining on code in c and e, also join on year in e and p.

In [12]:
sql ='''
-- 6. Select fields
SELECT c.code, name, region, e.year, fertility_rate, unemployment_rate
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join to populations (as p)
  INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
  -- 4. Join to economies (as e)
  INNER JOIN economies AS e
    -- 5. Match on country code and year
    ON c.code = e.code AND p.year = e.year;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,region,year,fertility_rate,unemployment_rate
0,AFG,Afghanistan,Southern and Central Asia,2010,5.746,NaN
1,AFG,Afghanistan,Southern and Central Asia,2015,4.653,NaN
2,AGO,Angola,Central Africa,2010,6.416,NaN
3,AGO,Angola,Central Africa,2015,5.996,NaN
4,ALB,Albania,Southern Europe,2010,1.663,14.0


## Inner join with using

When joining tables with a common field name, e.g.

You can use USING as a shortcut:

You'll now explore how this can be done with the countries and languages tables.

- Inner join countries on the left and languages on the right with USING(code).
- Select the fields corresponding to:
    - country name AS country,
    - continent name,
    - language name AS language, and
    - whether or not the language is official.
    
Remember to alias your tables using the first letter of their names.

In [16]:
sql ='''
-- 4. Select fields
SELECT c.name AS country, c.continent, l.name AS language, l.official
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join to languages (as l)
  INNER JOIN languages AS l
    -- 3. Match using code
    USING(code);
'''
df = pd.read_sql(sql, conn)
df.head()

,country,continent,language,official
0,Afghanistan,Asia,Dari,True
1,Afghanistan,Asia,Pashto,True
2,Afghanistan,Asia,Turkic,False
3,Afghanistan,Asia,Other,False
4,Albania,Europe,Albanian,True


## Self-join

In this exercise, you'll use the populations table to perform a self-join to calculate the percentage increase in population from 2010 to 2015 for each country code!

Since you'll be joining the populations table to itself, you can alias populations as p1 and also populations as p2. This is good practice whenever you are aliasing and your tables have the same first letter. Note that you are required to alias the tables with self-joins.

- Join populations with itself ON country_code.
- Select the country_code from p1 and the size field from both p1 and p2. SQL won't allow same-named fields, so alias p1.size as size2010 and p2.size as size2015.

In [17]:
sql ='''
-- 4. Select fields with aliases
SELECT p1.country_code, 
       p1.size AS size2010,
       p2.size AS size2015
-- 1. From populations (alias as p1)
FROM populations AS p1
  -- 2. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    -- 3. Match on country code
    ON  p1.country_code = p2.country_code;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size2010,size2015
0,ABW,101597.0,103889.0
1,ABW,101597.0,101597.0
2,ABW,103889.0,103889.0
3,ABW,103889.0,101597.0
4,AFG,27962200.0,32526600.0


Notice from the result that for each country_code you have four entries laying out all combinations of 2010 and 2015.

- Extend the ON in your query to include only those records where the p1.year (2010) matches with p2.year - 5 (2015 - 5 = 2010). This will omit the three entries per country_code that you aren't interested in.

In [18]:
sql ='''
-- 5. Select fields with aliases
SELECT p1.country_code,
       p1.size AS size2010,
       p2.size AS size2015
-- 1. From populations (alias as p1)
FROM populations as p1
  -- 2. Join to itself (alias as p2)
  INNER JOIN populations as p2
    -- 3. Match on country code
    ON p1.country_code = p2.country_code
        -- 4. and year (with calculation)
        AND p1.year = p2.year - 5;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size2010,size2015
0,ABW,101597.0,103889.0
1,AFG,27962200.0,32526600.0
2,AGO,21220000.0,25022000.0
3,ALB,2913020.0,2889170.0
4,AND,84419.0,70473.0


As you just saw, you can also use SQL to calculate values like p2.year - 5 for you. With two fields like size2010 and size2015, you may want to determine the percentage increase from one field to the next:

With two numeric fields A and B, the percentage growth from A to B can be calculated as (B−A)/A∗100.0.

Add a new field to SELECT, aliased as growth_perc, that calculates the percentage population growth from 2010 to 2015 for each country, using p2.size and p1.size.

In [19]:
sql ='''
SELECT p1.country_code,
       p1.size AS size2010, 
       p2.size AS size2015,
       -- 1. calculate growth_perc
       ((p2.size - p1.size)/p1.size * 100.0) AS growth_perc
-- 2. From populations (alias as p1)
FROM populations AS p1
  -- 3. Join to itself (alias as p2)
  INNER JOIN populations AS p2
    -- 4. Match on country code
    ON p1.country_code = p2.country_code
        -- 5. and year (with calculation)
        AND p1.year = p2.year - 5;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size2010,size2015,growth_perc
0,ABW,101597.0,103889.0,2.255972
1,AFG,27962200.0,32526600.0,16.323297
2,AGO,21220000.0,25022000.0,17.917192
3,ALB,2913020.0,2889170.0,-0.818875
4,AND,84419.0,70473.0,-16.519977


## Case when and then

Often it's useful to look at a numerical field not as raw data, but instead as being in different categories or groups.

You can use `CASE` with `WHEN`, `THEN`, `ELSE`, and `END` to define a new grouping field.

Using the countries table, create a new field AS geosize_group that groups the countries into three groups:

- If surface_area is greater than 2 million, geosize_group is 'large'.
- If surface_area is greater than 350 thousand but not larger than 2 million, geosize_group is 'medium'.
- Otherwise, geosize_group is 'small'.

In [20]:
sql ='''
SELECT name, continent, code, surface_area,
    -- 1. First case
    CASE WHEN surface_area > 2000000 THEN 'large'
        -- 2. Second case
        WHEN surface_area > 350000 THEN 'medium'
        -- 3. Else clause + end
        ELSE 'small' END
        -- 4. Alias name
        AS geosize_group
-- 5. From table
FROM countries;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,continent,code,surface_area,geosize_group
0,Afghanistan,Asia,AFG,652090.0,medium
1,Netherlands,Europe,NLD,41526.0,small
2,Albania,Europe,ALB,28748.0,small
3,Algeria,Africa,DZA,2381740.0,large
4,American Samoa,Oceania,ASM,199.0,small


## Inner challenge

The table you created with the added geosize_group field has been loaded for you here with the name countries_plus. Observe the use of (and the placement of) the INTO command to create this countries_plus table:

You will now explore the relationship between the size of a country in terms of surface area and in terms of population using grouping fields created with CASE.

By the end of this exercise, you'll be writing two queries back-to-back in a single script. You got this!

Using the populations table focused only for the year 2015, create a new field AS popsize_group to organize population size into

- 'large' (> 50 million),
- 'medium' (> 1 million), and
- 'small' groups.

Select only the country code, population size, and this new popsize_group as fields.

In [21]:
sql ='''
SELECT country_code, size,
    -- 1. First case
    CASE WHEN size > 50000000 THEN 'large'
        -- 2. Second case
        WHEN size > 1000000 THEN 'medium'
        -- 3. Else clause + end
        ELSE 'small' END
        -- 4. Alias name
        AS popsize_group
-- 5. From table
FROM populations
-- 6. Focus on 2015
WHERE year = 2015;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size,popsize_group
0,ABW,103889.0,small
1,AFG,32526600.0,medium
2,AGO,25022000.0,medium
3,ALB,2889170.0,medium
4,AND,70473.0,small


- Use INTO to save the result of the previous query as pop_plus. You can see an example of this in the countries_plus code in the assignment text. Make sure to include a ; at the end of your WHERE clause!

- Then, include another query below your first query to display all the records in pop_plus using SELECT * FROM pop_plus; so that you generate results and this will display pop_plus in query result.

In [22]:
sql ='''
SELECT country_code, size,
    CASE WHEN size > 50000000 THEN 'large'
        WHEN size > 1000000 THEN 'medium'
        ELSE 'small' END
        AS popsize_group
-- 1. Into table
INTO pop_plus
FROM populations
WHERE year = 2015;

-- 2. Select all columns of pop_plus
SELECT * FROM pop_plus;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code,size,popsize_group
0,ABW,103889.0,small
1,AFG,32526600.0,medium
2,AGO,25022000.0,medium
3,ALB,2889170.0,medium
4,AND,70473.0,small


- Keep the first query intact that creates pop_plus using INTO.
- Write a query to join countries_plus AS c on the left with pop_plus AS p on the right matching on the country code fields.
- Sort the data based on geosize_group, in ascending order so that large appears on top.
- Select the name, continent, geosize_group, and popsize_group fields.

In [24]:
sql ='''
SELECT country_code, size,
  CASE WHEN size > 50000000
            THEN 'large'
       WHEN size > 1000000
            THEN 'medium'
       ELSE 'small' END
       AS popsize_group
INTO pop_plus2       
FROM populations
WHERE year = 2015;

-- 5. Select fields
SELECT c.name, c.continent, c.geosize_group, p.popsize_group
-- 1. From countries_plus (alias as c)
FROM countries_plus as c
  -- 2. Join to pop_plus (alias as p)
  INNER JOIN pop_plus2 as p
    -- 3. Match on country code
    ON c.code = p.country_code
-- 4. Order the table    
ORDER BY c.geosize_group;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,continent,geosize_group,popsize_group
0,India,Asia,large,large
1,United States,North America,large,large
2,Saudi Arabia,Asia,large,medium
3,China,Asia,large,large
4,Kazakhstan,Asia,large,medium


## Left Join

<img src="images/sql-join-left.png" alt="" style="width: 250px;"/> <img src="images/sql-join-left2.png" alt="" style="width: 250px;"/>

Now you'll explore the differences between performing an `inner join` and a `left join` using the cities and countries tables.

You'll begin by performing an inner join with the cities table on the left and the countries table on the right. Remember to alias the name of the city field as city and the name of the country field as country.

You will then change the query to a left join. Take note of how many records are in each query here!

In [25]:
sql ='''
-- Select the city name (with alias), the country code,
-- the country name (with alias), the region,
-- and the city proper population
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
-- From left table (with alias)
FROM cities AS c1
  -- Join to right table (with alias)
  INNER JOIN countries AS c2
    -- Match on country code
    ON c1.country_code = c2.code
-- Order by descending country code
ORDER BY code DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,city,code,country,region,city_proper_pop
0,Harare,ZWE,Zimbabwe,Eastern Africa,1606000.0
1,Lusaka,ZMB,Zambia,Eastern Africa,1742980.0
2,Cape Town,ZAF,South Africa,Southern Africa,3740030.0
3,Johannesburg,ZAF,South Africa,Southern Africa,4434830.0
4,Durban,ZAF,South Africa,Southern Africa,3442360.0


Change the code to perform a LEFT JOIN instead of an INNER JOIN. After executing this query, note how many records the query result contains.

In [26]:
sql ='''
SELECT c1.name AS city, code, c2.name AS country,
       region, city_proper_pop
FROM cities AS c1
  -- 1. Join right table (with alias)
  LEFT JOIN countries AS c2
    -- 2. Match on country code
    ON c1.country_code = c2.code
-- 3. Order by descending country code
ORDER BY code DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,city,code,country,region,city_proper_pop
0,Taichung,None,None,None,2752410.0
1,Tainan,None,None,None,1885250.0
2,Kaohsiung,None,None,None,2778920.0
3,Bucharest,None,None,None,1883420.0
4,Taipei,None,None,None,2704970.0


Notice that the INNER JOIN version resulted in 230 records. The LEFT JOIN version returned 236 rows.

## Left join (2)

<img src="images/sql-join-inner-left.png" alt="" style="width: 600px;"/>

Next, you'll try out another example comparing an inner join to its corresponding left join. Before you begin though, take note of how many records are in both the countries and languages tables below.

You will begin with an inner join on the countries table on the left with the languages table on the right. Then you'll change the code to a left join in the next bullet.

Note the use of multi-line comments here using /* and */.

- Perform an inner join. Alias the name of the country field as country and the name of the language field as language.
- Sort based on descending country name.

In [27]:
sql ='''
/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  INNER JOIN languages AS l
    -- 3. Match on fields
    ON c.code = l.code
-- 4. Order by descending country
ORDER BY country DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,local_name,language,percent
0,Zimbabwe,Zimbabwe,Shona,NaN
1,Zimbabwe,Zimbabwe,Tonga,NaN
2,Zimbabwe,Zimbabwe,Tswana,NaN
3,Zimbabwe,Zimbabwe,Venda,NaN
4,Zimbabwe,Zimbabwe,Xhosa,NaN


- Perform a left join instead of an inner join. Observe the result, and also note the change in the number of records in the result.
- Carefully review which records appear in the left join result, but not in the inner join result.

In [28]:
sql ='''
/*
5. Select country name AS country, the country's local name,
the language name AS language, and
the percent of the language spoken in the country
*/
SELECT c.name AS country, local_name, l.name AS language, percent
-- 1. From left table (alias as c)
FROM countries AS c
  -- 2. Join to right table (alias as l)
  LEFT JOIN languages AS l
    -- 3. Match on fields
    ON c.code = l.code
-- 4. Order by descending country
ORDER BY country DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,local_name,language,percent
0,Zimbabwe,Zimbabwe,Chibarwe,NaN
1,Zimbabwe,Zimbabwe,Shona,NaN
2,Zimbabwe,Zimbabwe,Ndebele,NaN
3,Zimbabwe,Zimbabwe,English,NaN
4,Zimbabwe,Zimbabwe,Chewa,NaN


Notice that the INNER JOIN version resulted in 914 records. The LEFT JOIN version returned 921 rows.

## Left join (3)

You'll now revisit the use of the `AVG()` function. You will use it in combination with left join to determine the average gross domestic product (GDP) per capita by region in 2010.

- Begin with a left join with the countries table on the left and the economies table on the right.
- Focus only on records with 2010 as the year.

In [29]:
sql ='''
-- 5. Select name, region, and gdp_percapita
SELECT name, region, gdp_percapita
-- 1. From countries (alias as c)
FROM countries AS c
  -- 2. Left join with economies (alias as e)
  LEFT JOIN economies AS e
    -- 3. Match on code fields
    ON c.code = e.code
-- 4. Focus on 2010
WHERE e.year = 2010;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,region,gdp_percapita
0,Afghanistan,Southern and Central Asia,539.667
1,Angola,Central Africa,3599.270
2,Albania,Southern Europe,4098.130
3,United Arab Emirates,Middle East,34628.600
4,Argentina,South America,10413.000


- Modify your code to calculate the average GDP per capita AS avg_gdp for each region in 2010.
- Select the region and avg_gdp fields.

In [30]:
sql ='''
-- Select fields
SELECT region, AVG(gdp_percapita) as avg_gdp
-- From countries (alias as c)
FROM countries AS c
  -- Left join with economies (alias as e)
  LEFT JOIN economies AS e
    -- Match on code fields
    ON c.code = e.code
-- Focus on 2010
WHERE year = 2010
-- Group by region
GROUP BY region;
'''
df = pd.read_sql(sql, conn)
df.head()

,region,avg_gdp
0,Southern Africa,5051.597974
1,Caribbean,11413.339454
2,Eastern Africa,1757.348162
3,Southern Europe,22926.410911
4,Eastern Asia,26205.851400


Arrange this data on average GDP per capita for each region in 2010 from highest to lowest average GDP per capita.

In [31]:
sql ='''
-- Select fields
SELECT region, AVG(gdp_percapita) AS avg_gdp
-- From countries (alias as c)
FROM countries AS c
  -- Left join with economies (alias as e)
  LEFT JOIN economies AS e
    -- Match on code fields
    ON c.code = e.code
-- Focus on 2010
WHERE year = 2010
-- Group by region
GROUP BY region
-- Order by descending avg_gdp
ORDER BY avg_gdp DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,region,avg_gdp
0,Western Europe,58130.961496
1,Nordic Countries,57073.997656
2,North America,47911.509766
3,Australia and New Zealand,44792.384766
4,British Islands,43588.330078


## Right join

<img src="images/sql-join-right.png" alt="" style="width: 250px;"/>

<img src="images/sql-join-left-right.png" alt="" style="width: 600px;"/>

Right joins aren't as common as left joins. One reason why is that you can always write a right join as a left join.

The left join code is commented out here. Your task is to write a new query using rights joins that produces the same result as what the query using left joins produces. Keep this left joins code commented as you write your own query just below it using right joins to solve the problem.

Note the order of the joins matters in your conversion to using right joins!

In [32]:
sql ='''
-- convert this code to use RIGHT JOINs instead of LEFT JOINs
/*
SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM cities
  LEFT JOIN countries
    ON cities.country_code = countries.code
  LEFT JOIN languages
    ON countries.code = languages.code
ORDER BY city, language;
*/

SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM languages
  RIGHT JOIN countries
    ON countries.code = languages.code
  RIGHT JOIN cities
    ON cities.country_code = countries.code
ORDER BY city, language;
'''
df = pd.read_sql(sql, conn)
df.head()

,city,urbanarea_pop,country,indep_year,language,percent
0,Abidjan,4765000.0,Cote d'Ivoire,1960.0,French,NaN
1,Abidjan,4765000.0,Cote d'Ivoire,1960.0,Other,NaN
2,Abu Dhabi,1145000.0,United Arab Emirates,1971.0,Arabic,NaN
3,Abu Dhabi,1145000.0,United Arab Emirates,1971.0,English,NaN
4,Abu Dhabi,1145000.0,United Arab Emirates,1971.0,Hindi,NaN


## Full join

<img src="images/sql-join-full.png" alt="" style="width: 250px;"/>

In this exercise, you'll examine how your results differ when using a full join versus using a left join versus using an inner join with the countries and currencies tables.

You will focus on the North American region and also where the name of the country is missing. Dig in to see what we mean!

Begin with a full join with countries on the left and currencies on the right. The fields of interest have been SELECTed for you throughout this exercise.

Then complete a similar left join and conclude with an inner join.

Choose records in which region corresponds to North America or is NULL.

In [33]:
sql ='''
SELECT name AS country, code, region, basic_unit
-- 3. From countries
FROM countries
  -- 4. Join to currencies
  FULL JOIN currencies
    -- 5. Match on code
    USING (code)
-- 1. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
-- 2. Order by region
ORDER BY region;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,code,region,basic_unit
0,Canada,CAN,North America,Canadian dollar
1,United States,USA,North America,United States dollar
2,Bermuda,BMU,North America,Bermudian dollar
3,Greenland,GRL,North America,None
4,None,TMP,None,United States dollar


Repeat the same query as above but use a LEFT JOIN instead of a FULL JOIN. Note what has changed compared to the FULL JOIN result!

In [34]:
sql ='''
SELECT name AS country, code, region, basic_unit
-- 1. From countries
FROM countries
  -- 2. Join to currencies
  LEFT JOIN currencies
    -- 3. Match on code
    USING (code)
-- 4. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
-- 5. Order by region
ORDER BY region;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,code,region,basic_unit
0,Bermuda,BMU,North America,Bermudian dollar
1,Canada,CAN,North America,Canadian dollar
2,United States,USA,North America,United States dollar
3,Greenland,GRL,North America,None


Repeat the same query as above but use an INNER JOIN instead of a FULL JOIN. Note what has changed compared to the FULL JOIN and LEFT JOIN results!

In [35]:
sql ='''
SELECT name AS country, code, region, basic_unit
FROM countries
  -- 1. Join to currencies
  INNER JOIN currencies
    USING (code)
-- 2. Where region is North America or null
WHERE region = 'North America' OR region IS NULL
-- 3. Order by region
ORDER BY region;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,code,region,basic_unit
0,Bermuda,BMU,North America,Bermudian dollar
1,Canada,CAN,North America,Canadian dollar
2,United States,USA,North America,United States dollar


Have you kept an eye out on the different numbers of records these queries returned? The FULL JOIN query returned 17 rows, the OUTER JOIN returned 4 rows, and the INNER JOIN only returned 3 rows. Do these results make sense to you?

## Full join (2)

You'll now investigate a similar exercise to the last one, but this time focused on using a table with more records on the left than the right. You'll work with the languages and countries tables.

Begin with a full join with languages on the left and countries on the right. Appropriate fields have been selected for you again here.

- Choose records in which countries.name starts with the capital letter 'V' or is NULL.
- Arrange by countries.name in ascending order to more clearly see the results.

In [36]:
sql ='''
SELECT countries.name, code, languages.name AS language
-- 3. From languages
FROM languages
  -- 4. Join to countries
  FULL JOIN countries
    -- 5. Match on code
    USING (code)
-- 1. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
-- 2. Order by ascending countries.name
ORDER BY countries.name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,code,language
0,Vanuatu,VUT,Tribal Languages
1,Vanuatu,VUT,English
2,Vanuatu,VUT,French
3,Vanuatu,VUT,Other
4,Vanuatu,VUT,Bislama


Repeat the same query as above but use a left join instead of a full join. Note what has changed compared to the full join result!

In [37]:
sql ='''
SELECT countries.name, code, languages.name AS language
FROM languages
  -- 1. Join to countries
  LEFT JOIN countries
    -- 2. Match using code
    USING (code)
-- 3. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
ORDER BY countries.name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,code,language
0,Vanuatu,VUT,English
1,Vanuatu,VUT,Other
2,Vanuatu,VUT,French
3,Vanuatu,VUT,Tribal Languages
4,Vanuatu,VUT,Bislama


Repeat once more, but use an inner join instead of a left join. Note what has changed compared to the full join and left join results.

In [38]:
sql ='''
SELECT countries.name, code, languages.name AS language
FROM languages
  -- 1. Join to countries
  INNER JOIN countries
    USING (code)
-- 2. Where countries.name starts with V or is null
WHERE countries.name LIKE 'V%' OR countries.name IS NULL
ORDER BY countries.name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,code,language
0,Vanuatu,VUT,Tribal Languages
1,Vanuatu,VUT,Bislama
2,Vanuatu,VUT,English
3,Vanuatu,VUT,French
4,Vanuatu,VUT,Other


## Full join (3)

You'll now explore using two consecutive full joins on the three tables you worked with in the previous two exercises.

- Complete a full join with countries on the left and languages on the right.
- Next, full join this result with currencies on the right.
- Use LIKE to choose the Melanesia and Micronesia regions (Hint: 'M%esia').
- Select the fields corresponding to the country name AS country, region, language name AS language, and basic and fractional units of currency.

In [39]:
sql ='''
-- 7. Select fields (with aliases)
SELECT c1.name AS country, region, l.name AS language,
       basic_unit, frac_unit
-- 1. From countries (alias as c1)
FROM countries AS c1
  -- 2. Join with languages (alias as l)
  FULL JOIN languages AS l
    -- 3. Match on code
    USING (code)
  -- 4. Join with currencies (alias as c2)
  FULL JOIN currencies AS c2
    -- 5. Match on code
    USING (code)
-- 6. Where region like Melanesia and Micronesia
WHERE region LIKE 'M%esia';
'''
df = pd.read_sql(sql, conn)
df.head()

,country,region,language,basic_unit,frac_unit
0,Kiribati,Micronesia,English,Australian dollar,Cent
1,Kiribati,Micronesia,Kiribati,Australian dollar,Cent
2,Marshall Islands,Micronesia,Other,United States dollar,Cent
3,Marshall Islands,Micronesia,Marshallese,United States dollar,Cent
4,Nauru,Micronesia,Other,Australian dollar,Cent


## CROSS JOIN - A table of two cities

<img src="images/sql-join-cross.png" alt="" style="width: 200px;"/>

This exercise looks to explore languages potentially and most frequently spoken in the cities of Hyderabad, India and Hyderabad, Pakistan.

You will begin with a cross join with cities AS c on the left and languages AS l on the right. Then you will modify the query using an inner join in the next tab.

- Create the cross join as described above. (Recall that cross joins do not use ON or USING.)
- Make use of LIKE and Hyder% to choose Hyderabad in both countries.
- Select only the city name AS city and language name AS language.

In [40]:
sql ='''
-- 4. Select fields
SELECT c.name AS city, l.name AS language
-- 1. From cities (alias as c)
FROM cities AS c        
  -- 2. Join to languages (alias as l)
  CROSS JOIN languages AS l
-- 3. Where c.name like Hyderabad
WHERE c.name LIKE 'Hyder%';
'''
df = pd.read_sql(sql, conn)
df.head()

,city,language
0,Hyderabad (India),Dari
1,Hyderabad,Dari
2,Hyderabad (India),Pashto
3,Hyderabad,Pashto
4,Hyderabad (India),Turkic


- Use an inner join instead of a cross join. Think about what the difference will be in the results for this inner join result and the one for the cross join.

In [41]:
sql ='''
-- 5. Select fields
SELECT c.name AS city, l.name AS language
-- 1. From cities (alias as c)
FROM cities AS c      
  -- 2. Join to languages (alias as l)
  INNER JOIN languages AS l
    -- 3. Match on country code
    ON c.country_code = l.code
-- 4. Where c.name like Hyderabad
WHERE c.name LIKE 'Hyder%';
'''
df = pd.read_sql(sql, conn)
df.head()

,city,language
0,Hyderabad (India),Hindi
1,Hyderabad (India),Bengali
2,Hyderabad (India),Telugu
3,Hyderabad (India),Marathi
4,Hyderabad (India),Tamil


## Outer challenge

Now that you're fully equipped to use outer joins, try a challenge problem to test your knowledge!

In terms of life expectancy for 2010, determine the names of the lowest five countries and their regions.

- Select country name AS country, region, and life expectancy AS life_exp.
- Make sure to use LEFT JOIN, WHERE, ORDER BY, and LIMIT.

In [42]:
sql ='''
-- Select fields
SELECT c.name as country, region, p.life_expectancy as life_exp
-- From countries (alias as c)
FROM countries AS c
  -- Join to populations (alias as p)
  LEFT JOIN populations AS p
    -- Match on country code
    ON c.code = p.country_code
-- Focus on 2010
WHERE p.year = 2010
-- Order by life_exp
ORDER BY life_exp
-- Limit to 5 records
LIMIT 5;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,region,life_exp
0,Lesotho,Southern Africa,47.4834
1,Central African Republic,Central Africa,47.6253
2,Sierra Leone,Western Africa,48.2290
3,Swaziland,Southern Africa,48.3458
4,Zimbabwe,Eastern Africa,49.5747


## Set Theory Venn Diagrams

<img src="images/sql-set-theory-venn-diagrams.png" alt="" style="width: 400px;"/>


## Union

<img src="images/sql-union.png" alt="" style="width: 200px;"/>

You will see two new tables with names economies2010 and economies2015. 

- Combine these two tables into one table containing all of the fields in economies2010. The economies table is also included for reference.
- Sort this resulting single table by country code and then by year, both in ascending order.

In [43]:
sql ='''
-- Select fields from 2010 table
SELECT *
  -- From 2010 table
  FROM economies2010
	-- Set theory clause
	UNION
-- Select fields from 2015 table
SELECT *
  -- From 2015 table
  FROM economies2015
-- Order by code and year
ORDER BY code, year;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,year,income_group,gross_savings
0,AFG,2010,Low income,37.133
1,AFG,2015,Low income,21.466
2,AGO,2010,Upper middle income,23.534
3,AGO,2015,Upper middle income,-0.425
4,ALB,2010,Upper middle income,20.011


## Union (2)

`UNION` can also be used to determine all occurrences of a field across multiple tables. Try out this exercise with no starter code.

- Determine all (non-duplicated) country codes in either the cities or the currencies table. The result should be a table with only one field called country_code.
- Sort by country_code in alphabetical order.

In [44]:
sql ='''
-- Select field
SELECT country_code
  -- From cities
  FROM cities
	-- Set theory clause
	UNION
-- Select field
SELECT code as country_code
  -- From currencies
  FROM currencies
-- Order by country_code
ORDER BY country_code;
'''
df = pd.read_sql(sql, conn)
df.head()

,country_code
0,ABW
1,AFG
2,AGO
3,AIA
4,ALB


## Union all

<img src="images/sql-union-all.png" alt="" style="width: 200px;"/>

As you saw, duplicates were removed from the previous two exercises by using UNION.

To include duplicates, you can use `UNION ALL`.

- Determine all combinations (include duplicates) of country code and year that exist in either the economies or the populations tables. Order by code then year.
- The result of the query should only have two columns/fields. Think about how many records this query should result in.

In [45]:
sql ='''
-- Select fields
SELECT code, year
  -- From economies
  FROM economies
	-- Set theory clause
	UNION ALL
-- Select fields
SELECT country_code, year
  -- From populations
  FROM populations
-- Order by code, year
ORDER BY code, year;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,year
0,ABW,2010
1,ABW,2015
2,AFG,2010
3,AFG,2010
4,AFG,2015


## Intersect

<img src="images/sql-intersect.png" alt="" style="width: 250px;"/>

Repeat the previous UNION ALL exercise, this time looking at the records in common for country code and year for the economies and populations tables.

- Again, order by code and then by year, both in ascending order.
- Note the number of records here (given at the bottom of query result) compared to the similar UNION ALL query result (814 records).

In [47]:
sql ='''
-- Select fields
SELECT code, year
  -- From economies
  FROM economies
	-- Set theory clause
	INTERSECT
-- Select fields
SELECT country_code, year
  -- From populations
  FROM populations
-- Order by code and year
ORDER BY code, year;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,year
0,AFG,2010
1,AFG,2015
2,AGO,2010
3,AGO,2015
4,ALB,2010


## Intersect (2)

As you think about major world cities and their corresponding country, you may ask which countries also have a city with the same name as their country name? Use INTERSECT to answer this question with countries and cities!

In [48]:
sql ='''
-- Select fields
SELECT name
  -- From countries
  FROM countries
	-- Set theory clause
	INTERSECT
-- Select fields
SELECT name
  -- From cities
  FROM cities;
'''
df = pd.read_sql(sql, conn)
df.head()

,name
0,Singapore
1,Hong Kong


Hong Kong is part of China, but it appears separately here because it has its own ISO country code. Depending upon your analysis, treating Hong Kong separately could be useful or a mistake. Always check your dataset closely before you perform an analysis!

## Except

<img src="images/sql-except.png" alt="" style="width: 250px;"/>

Get the names of cities in cities which are not noted as capital cities in countries as a single field result.

Note that there are some countries in the world that are not included in the countries table, which will result in some cities not being labeled as capital cities when in fact they are.

- Order the resulting field in ascending order.
- Can you spot the city/cities that are actually capital cities which this query misses?

In [49]:
sql ='''
-- Select field
SELECT city.name
  -- From cities
  FROM cities AS city
	-- Set theory clause
	EXCEPT
-- Select field
SELECT country.capital
  -- From countries
  FROM countries AS country
-- Order by result
ORDER BY name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name
0,Abidjan
1,Ahmedabad
2,Alexandria
3,Almaty
4,Auckland


## Except (2)

Now you will complete the previous query in reverse!

Determine the names of capital cities that are not listed in the cities table.

- Order by capital in ascending order.
- The cities table contains information about 236 of the world's most populous cities. The result of your query may surprise you in terms of the number of capital cities that DO NOT appear in this list!

In [50]:
sql ='''
-- Select field
SELECT country.capital
  -- From countries
  FROM countries AS country
	-- Set theory clause
	EXCEPT
-- Select field
SELECT city.name
  -- From cities
  FROM cities AS city
-- Order by ascending capital
ORDER BY capital;
'''
df = pd.read_sql(sql, conn)
df.head()

,capital
0,Agana
1,Amman
2,Amsterdam
3,Andorra la Vella
4,Antananarivo


## Semi-join

<img src="images/sql-join-semi-anti.png" alt="" style="width: 600px;"/>

You are now going to use the concept of a semi-join to identify languages spoken in the Middle East.

Comment out the answer to the previous tab by surrounding it in /* and */. You'll come back to it!
Below the commented code, select only unique languages by name appearing in the languages table.
Order the resulting single field table by name in ascending order.

In [51]:
sql ='''
SELECT code
  FROM countries
WHERE region = 'Middle East';

-- Select field
SELECT DISTINCT name
  -- From languages
  FROM languages
-- Order by name
ORDER BY name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name
0,Afar
1,Afrikaans
2,Akyem
3,Albanian
4,Alsatian


In [52]:
sql ='''
-- Select distinct fields
SELECT DISTINCT name
  -- From languages
  FROM languages
-- Where in statement
WHERE code IN
  -- Subquery
  (SELECT code 
        FROM countries
        WHERE region='Middle East')
-- Order by name
ORDER BY name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name
0,Arabic
1,Aramaic
2,Armenian
3,Azerbaijani
4,Azeri


## Diagnosing problems using anti-join

Another powerful join in SQL is the anti-join. It is particularly useful in identifying which records are causing an incorrect number of records to appear in join queries.

You will also see another example of a subquery here, as you saw in the first exercise on semi-joins. Your goal is to identify the currencies used in Oceanian countries!

- Begin by determining the number of countries in countries that are listed in Oceania using SELECT, FROM, and WHERE.

In [53]:
sql ='''
-- Select statement
SELECT COUNT(*)
  -- From countries
  FROM countries
-- Where continent is Oceania
WHERE continent = 'Oceania'; 
'''
df = pd.read_sql(sql, conn)
df.head()

,count
0,19


- Complete an inner join with countries AS c1 on the left and currencies AS c2 on the right to get the different currencies used in the countries of Oceania.
- Match ON the code field in the two tables.
- Include the country code, country name, and basic_unit AS currency.
- Observe query result and make note of how many different countries are listed here.

In [54]:
sql ='''
-- 5. Select fields (with aliases)
SELECT c1.code, c1.name, c2.basic_unit AS currency
  -- 1. From countries (alias as c1)
  FROM countries AS c1
  	-- 2. Join with currencies (alias as c2)
  	INNER JOIN currencies AS c2
    -- 3. Match on code
    USING (code)
-- 4. Where continent is Oceania
WHERE continent = 'Oceania';
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name,currency
0,AUS,Australia,Australian dollar
1,PYF,French Polynesia,CFP franc
2,KIR,Kiribati,Australian dollar
3,MHL,Marshall Islands,United States dollar
4,NRU,Nauru,Australian dollar


Note that not all countries in Oceania were listed in the resulting inner join with currencies. Use an anti-join to determine which countries were not included!

- Use NOT IN and (SELECT code FROM currencies) as a subquery to get the country code and country name for the Oceanian countries that are not included in the currencies table.

In [55]:
sql ='''
-- 3. Select fields
SELECT c1.code, c1.name
  -- 4. From Countries
  FROM countries AS c1
  -- 5. Where continent is Oceania
  WHERE c1.continent = 'Oceania'
  	-- 1. And code not in
  	AND code NOT IN
  	-- 2. Subquery
  	(SELECT code 
    FROM currencies);
'''
df = pd.read_sql(sql, conn)
df.head()

,code,name
0,ASM,American Samoa
1,FJI,Fiji Islands
2,GUM,Guam
3,FSM,"Micronesia, Federated States of"
4,MNP,Northern Mariana Islands


## Set theory challenge

Incorporate two of UNION/UNION ALL/INTERSECT/EXCEPT to solve a challenge involving three tables.

- Identify the country codes that are included in either economies or currencies but not in populations.
- Use that result to determine the names of cities in the countries that match the specification in the previous instruction.

In [56]:
sql ='''
-- Select the city name
SELECT name
  -- Alias the table where city name resides
  FROM cities AS c1
  -- Choose only records matching the result of multiple set theory clauses
  WHERE country_code IN
(
    -- Select appropriate field from economies AS e
    SELECT e.code
    FROM economies AS e
    -- Get all additional (unique) values of the field from currencies AS c2  
    UNION
    SELECT c2.code
    FROM currencies AS c2
    -- Exclude those appearing in populations AS p
    EXCEPT
    SELECT p.country_code
    FROM populations AS p
);
'''
df = pd.read_sql(sql, conn)
df.head()

,name
0,Bucharest
1,Kaohsiung
2,New Taipei City
3,Taichung
4,Tainan


## Subquery inside where

You'll now try to figure out which countries had high average life expectancies (at the country level) in 2015.

- Begin by calculating the average life expectancy across all countries for 2015.

In [57]:
sql ='''
-- Select average life_expectancy
SELECT AVG(life_expectancy)
  -- From populations
  FROM populations
-- Where year is 2015
WHERE year = 2015;
'''
df = pd.read_sql(sql, conn)
df.head()

,avg
0,71.676342


Recall that you can use SQL to do calculations for you. Suppose we wanted only records that were above `1.15 * 100` in terms of life expectancy for 2015:

Select all fields from populations with records corresponding to larger than 1.15 times the average you calculated in the first task for 2015. In other words, change the 100 in the example above with a subquery.

In [58]:
sql ='''
-- Select fields
SELECT * 
  -- From populations
  FROM populations
-- Where life_expectancy is greater than
WHERE life_expectancy > 1.15 * 
  -- 1.15 * subquery
  (SELECT AVG(life_expectancy)
    FROM populations
    WHERE year = 2015)
   AND year = 2015;
'''
df = pd.read_sql(sql, conn)
df.head()

,pop_id,country_code,year,fertility_rate,life_expectancy,size
0,21,AUS,2015,1.833,82.4512,23789800.0
1,376,CHE,2015,1.540,83.1976,8281430.0
2,356,ESP,2015,1.320,83.3805,46444000.0
3,134,FRA,2015,2.010,82.6707,66538400.0
4,170,HKG,2015,1.195,84.2780,7305700.0


## Subquery inside where (2)

Use your knowledge of subqueries in WHERE to get the urban area population for only capital cities.

- Make use of the capital field in the countries table in your subquery.
- Select the city name, country code, and urban area population fields.

In [59]:
sql ='''
-- 2. Select fields
SELECT city.name, city.country_code, city.urbanarea_pop
  -- 3. From cities
  FROM cities as city
-- 4. Where city name in the field of capital cities
WHERE city.name IN
  -- 1. Subquery
  (SELECT capital
   FROM countries)
ORDER BY urbanarea_pop DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,country_code,urbanarea_pop
0,Beijing,CHN,21516000.0
1,Dhaka,BGD,14543100.0
2,Tokyo,JPN,13513700.0
3,Moscow,RUS,12197600.0
4,Cairo,EGY,10230400.0


## Subquery inside select

In this exercise, you'll see how some queries can be written using either a join or a subquery.

You have seen previously how to use GROUP BY with aggregate functions and an inner join to get summarized information from multiple tables.

The code given in query.sql selects the top nine countries in terms of number of cities appearing in the cities table. Recall that this corresponds to the most populous cities in the world. Your task will be to convert the commented out code to get the same result as the code shown.

In [60]:
sql ='''
SELECT countries.name AS country, COUNT(*) AS cities_num
  FROM cities
    INNER JOIN countries
    ON countries.code = cities.country_code
GROUP BY country
ORDER BY cities_num DESC, country
LIMIT 9;

/* 
SELECT ___ AS ___,
  (SELECT ___
   FROM ___
   WHERE countries.code = cities.country_code) AS cities_num
FROM ___
ORDER BY ___ ___, ___
LIMIT 9;
*/
'''
df = pd.read_sql(sql, conn)
df.head()

,country,cities_num
0,China,36
1,India,18
2,Japan,11
3,Brazil,10
4,Pakistan,9


- Remove the comments around the second query and comment out the first query instead.
- Convert the GROUP BY code to use a subquery inside of SELECT, i.e. fill in the blanks to get a result that matches the one given using the GROUP BY code in the first query.
- Again, sort the result by cities_num descending and then by country ascending.

In [62]:
sql ='''
/*
SELECT countries.name AS country, COUNT(*) AS cities_num
  FROM cities
    INNER JOIN countries
    ON countries.code = cities.country_code
GROUP BY country
ORDER BY cities_num DESC, country
LIMIT 9;
*/
 
SELECT countries.name AS country,
  (SELECT COUNT(*)
   FROM cities
   WHERE countries.code = cities.country_code) AS cities_num
FROM countries
ORDER BY cities_num DESC, country
LIMIT 9;
'''
df = pd.read_sql(sql, conn)
df.head()

,country,cities_num
0,China,36
1,India,18
2,Japan,11
3,Brazil,10
4,Pakistan,9


## Subquery inside from

The last type of subquery you will work with is one inside of FROM.

You will use this to determine the number of languages spoken for each country, identified by the country's local name! (Note this may be different than the name field and is stored in the local_name field.)

- Begin by determining for each country code how many languages are listed in the languages table using SELECT, FROM, and GROUP BY.
- Alias the aggregated field as lang_num.

In [63]:
sql ='''
-- Select fields (with aliases)
SELECT code, COUNT(name) AS lang_num
  -- From languages
  FROM languages
-- Group by code
GROUP BY code;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,lang_num
0,PRY,2
1,NRU,3
2,MDG,3
3,ASM,5
4,TZA,4


- Include the previous query (aliased as subquery) as a subquery in the FROM clause of a new query.
- Select the local name of the country from countries.
- Also, select lang_num from subquery.
- Make sure to use WHERE appropriately to match code in countries and in subquery.
- Sort by lang_num in descending order.

In [64]:
sql ='''
-- Select fields
SELECT local_name, subquery.lang_num
  -- From countries
  FROM countries,
  	-- Subquery (alias as subquery)
  	(SELECT code, COUNT(name) AS lang_num
  	 FROM languages
    GROUP BY code) AS subquery
  -- Where codes match
  WHERE countries.code = subquery.code
-- Order by descending number of languages
ORDER BY lang_num DESC;
'''
df = pd.read_sql(sql, conn)
df.head()

,local_name,lang_num
0,Zambia,19
1,YeItyop´iya,16
2,Zimbabwe,16
3,Bharat/India,14
4,Nepal,14


## Advanced subquery
You can also nest multiple subqueries to answer even more specific questions.

In this exercise, for each of the six continents listed in 2015, you'll identify which country had the maximum inflation rate (and how high it was) using multiple subqueries. The table result of your query in Task 3 should look something like the following, where anything between < > will be filled in with appropriate values:

Again, there are multiple ways to get to this solution using only joins, but the focus here is on showing you an introduction into advanced subqueries.

- Create an inner join with countries on the left and economies on the right with USING. Do not alias your tables or columns.
- Retrieve the country name, continent, and inflation rate for 2015.

In [65]:
sql ='''
-- Select fields
SELECT name, continent, inflation_rate
  -- From countries
  FROM countries
  	-- Join to economies
  	INNER JOIN economies
    -- Match on code
    USING (code)
-- Where year is 2015
WHERE year = 2015;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,continent,inflation_rate
0,Afghanistan,Asia,-1.549
1,Angola,Africa,10.287
2,Albania,Europe,1.896
3,United Arab Emirates,Asia,4.070
4,Argentina,South America,NaN


- Determine the maximum inflation rate for each continent in 2015 using the previous query as a subquery called subquery in the FROM clause.
- Select the maximum inflation rate AS max_inf grouped by continent.
- This will result in the six maximum inflation rates in 2015 for the six continents as one field table. (Don't include continent in the outer SELECT statement.)

In [66]:
sql ='''
-- Select fields
SELECT MAX(inflation_rate) AS max_inf
  -- Subquery using FROM (alias as subquery)
  FROM (
      SELECT name, continent, inflation_rate
      FROM countries
      INNER JOIN economies
      USING (code)
      WHERE year = 2015) AS subquery
-- Group by continent
GROUP BY continent;
'''
df = pd.read_sql(sql, conn)
df.head()

,max_inf
0,21.858
1,39.403
2,121.738
3,7.524
4,48.684


- Append the second part's query to the first part's query using WHERE, AND, and IN to obtain the name of the country, its continent, and the maximum inflation rate for each continent in 2015. Revisit the sample output in the assignment text at the beginning of the exercise to see how this matches up.
- For the sake of practice, change all joining conditions to use ON instead of USING.
- This code works since each of the six maximum inflation rate values occur only once in the 2015 data. Think about whether this particular code involving subqueries would work in cases where there are ties for the maximum inflation rate values.

In [67]:
sql ='''
-- Select fields
SELECT name, continent, inflation_rate
  -- From countries
  FROM countries
	-- Join to economies
	INNER JOIN economies
	-- Match on code
	ON countries.code = economies.code
  -- Where year is 2015
  WHERE year = 2015
    -- And inflation rate in subquery (alias as subquery)
    AND inflation_rate IN (
        SELECT MAX(inflation_rate) AS max_inf
        FROM (
             SELECT name, continent, inflation_rate
             FROM countries
             INNER JOIN economies
             ON countries.code = economies.code
             WHERE year = 2015) AS subquery
        GROUP BY continent);
'''
df = pd.read_sql(sql, conn)
df.head()

,name,continent,inflation_rate
0,Haiti,North America,7.524
1,Malawi,Africa,21.858
2,Nauru,Oceania,9.784
3,Ukraine,Europe,48.684
4,Venezuela,South America,121.738


## Subquery challenge

Let's test your understanding of the subqueries with a challenge problem! Use a subquery to get 2015 economic data for countries that do not have

- gov_form of 'Constitutional Monarchy' or
- 'Republic' in their gov_form.

Here, gov_form stands for the form of the government for each country. Review the different entries for gov_form in the countries table.

- Select the country code, inflation rate, and unemployment rate.
- Order by inflation rate ascending.
- Do not use table aliasing in this exercise.

In [68]:
sql ='''
-- Select fields
SELECT code, inflation_rate, unemployment_rate
  -- From economies
  FROM economies
  -- Where year is 2015 and code is not in
  WHERE year = 2015 AND code NOT IN
  	-- Subquery
  	(SELECT code
   FROM countries
   WHERE (gov_form = 'Constitutional Monarchy' OR gov_form LIKE '%Republic'))
-- Order by inflation rate
ORDER BY inflation_rate;
'''
df = pd.read_sql(sql, conn)
df.head()

,code,inflation_rate,unemployment_rate
0,AFG,-1.549,NaN
1,CHE,-1.140,3.178
2,PRI,-0.751,12.000
3,ROU,-0.596,6.812
4,BRN,-0.423,6.900


## Final challenge

Read carefully over the instructions and solve them step-by-step, thinking about how the different clauses work together.

In this exercise, you'll need to get the country names and other 2015 data in the economies table and the countries table for Central American countries with an official language.

- Select unique country names. Also select the total investment and imports fields.
- Use a left join with countries on the left. (An inner join would also work, but please use a left join here.)
- Match on code in the two tables AND use a subquery inside of ON to choose the appropriate languages records.
- Order by country name ascending.
- Use table aliasing but not field aliasing in this exercise.

In [70]:
sql ='''
-- Select fields
SELECT DISTINCT c.name, e.total_investment, e.imports
  -- From table (with alias)
  FROM countries AS c
    -- Join with table (with alias)
    LEFT JOIN economies AS e
      -- Match on code
      ON (c.code = e.code 
      -- and code in Subquery
        AND c.code IN (SELECT code 
        FROM languages
        WHERE official = 'true') )
  -- Where region and year are correct
  WHERE year = 2015 AND region = 'Central America'
-- Order by field
ORDER BY c.name;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,total_investment,imports
0,Belize,22.014,6.743
1,Costa Rica,20.218,4.629
2,El Salvador,13.983,8.193
3,Guatemala,13.433,15.124
4,Honduras,24.633,9.353


## Final challenge (2)

Let's ease up a bit and calculate the average fertility rate for each region in 2015.

- Include the name of region, its continent, and average fertility rate aliased as avg_fert_rate.
- Sort based on avg_fert_rate ascending.
- Remember that you'll need to GROUP BY all fields that aren't included in the aggregate function of SELECT.

In [71]:
sql ='''
-- Select fields
SELECT region, continent, AVG(fertility_rate) AS avg_fert_rate
  -- From left table
  FROM countries AS c
    -- Join to right table
    INNER JOIN populations AS p
      -- Match on join condition
      ON c.code = p.country_code
  -- Where specific records matching some condition
  WHERE year = 2015
-- Group appropriately
GROUP BY region, continent
-- Order appropriately
ORDER BY avg_fert_rate;
'''
df = pd.read_sql(sql, conn)
df.head()

,region,continent,avg_fert_rate
0,Southern Europe,Europe,1.426100
1,Eastern Europe,Europe,1.490889
2,Baltic Countries,Europe,1.603333
3,Eastern Asia,Asia,1.620714
4,Western Europe,Europe,1.632500


## Final challenge (3)

You are now tasked with determining the top 10 capital cities in Europe and the Americas in terms of a calculated percentage using city_proper_pop and metroarea_pop in cities.

Do not use table aliasing in this exercise.

- Select the city name, country code, city proper population, and metro area population.
- Calculate the percentage of metro area population composed of city proper population for each city in cities, aliased as city_perc.
- Focus only on capital cities in Europe and the Americas in a subquery.
- Make sure to exclude records with missing data on metro area population.
- Order the result by city_perc descending.
- Then determine the top 10 capital cities in Europe and the Americas in terms of this city_perc percentage.

In [72]:
sql ='''
-- Select fields
SELECT name, country_code, city_proper_pop, metroarea_pop,  
      -- Calculate city_perc
      city_proper_pop / metroarea_pop * 100 AS city_perc
  -- From appropriate table
  FROM cities
  -- Where 
  WHERE name IN
    -- Subquery
    (SELECT capital
     FROM countries
     WHERE (continent = 'Europe'
      OR continent LIKE '%America'))
     AND metroarea_pop IS NOT NULL
-- Order appropriately
ORDER BY city_perc DESC
-- Limit amount
LIMIT 10;
'''
df = pd.read_sql(sql, conn)
df.head()

,name,country_code,city_proper_pop,metroarea_pop,city_perc
0,Lima,PER,8852000.0,10750000.0,82.344186
1,Bogota,COL,7878780.0,9800000.0,80.395746
2,Moscow,RUS,12197600.0,16170000.0,75.433493
3,Vienna,AUT,1863880.0,2600000.0,71.687728
4,Montevideo,URY,1305080.0,1947600.0,67.009616
